# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

#### Ler arquivos externos

In [2]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [3]:
def make_df_plen(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em plenário
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
        
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'     : [ file for file in file_list ],
        'CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE' : [ re.search("\-([A-Z\s]+)\-", file).group(1) for file in file_list ],
        'SESSION_DATE' : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

In [4]:
def make_df_com(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em comissão
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'     : [ file for file in file_list ],
        'CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE' : [ '-' for file in file_list ],
        'SESSION_DATE' : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

#### Funções de formatação e busca usando regex

In [5]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [6]:
def extract_full_quote(string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer de Jair Bolsonaro. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    '''
    
    if "O SR. PRESIDENTE (Jair Bolsonaro)" in string:
        pattern = "O SR\. PRESIDENTE (\(Jair Bolsonaro\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 1

    else:
        pattern  = "O?\s?SR\.?\s?(DEPUTADO)?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 3
        
    regexp = re.compile(pattern, re.MULTILINE)
    matches = re.findall(regexp, string)
    
    full_quote = [ ]
    for match in matches:
        quote = match[group_no]
        quote = ' '.join(quote.split()) # Remove espaços múltiplos internos usando a operação join de lista
        quote = quote.replace("- ", "")
        full_quote.append(quote)
        
    full_quote = ' [ INTERRUPÇÃO ] '.join(full_quote)
    return full_quote

#### Funções para aplicar operações de regex no dataframe

In [7]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [8]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_quote = extract_full_quote(row.CONTENT)
    return pd.Series({
        "FULL_QUOTE" : full_quote
    })

#### Função que encapsula anteriores e roda a operação

In [36]:
def run_script():
    
    '''
    Executa as operações anteriores em ambos os bancos de dados
    (plenário e comissões), filtra entradas sem match, concatena
    ambos os dataframes e salva para arquivo csv.
    '''
    
    # Monta dataframes
    jb_plen = make_df_plen( find_fpaths("../data/txts/plenario/", "*.txt") )
    jb_com  = make_df_com( find_fpaths("../data/txts/comissao/", "*.txt") )
    
    # As sessões do tipo HOMENAGEM são apenas registro de protocolo. 
    # Não contém transcrição de discursos.
    jb_plen = jb_plen[ jb_plen.SESSION_TYPE != "HOMENAGEM" ]
    
    # Aplica funções para extrair discursos
    jb_plen["SPEAKER_COUNT"] = jb_plen.apply(apply_count_speakers, axis=1)
    jb_plen["FULL_QUOTE"]    = jb_plen.apply(apply_extract_full_quote, axis=1)
    jb_com["SPEAKER_COUNT"]  = jb_com.apply(apply_count_speakers, axis=1)
    jb_com["FULL_QUOTE"]     = jb_com.apply(apply_extract_full_quote, axis=1)
    
    # Há sim entradas que não tiveram match algum
    # Verifiquei manualmente e elas são de discursos
    # que o site da Câmara retornou erroneamente.
    # De fato não contém falas de Bolsonaro.
    jb_plen = jb_plen[ jb_plen.FULL_QUOTE != "" ]
    jb_com  = jb_com[ jb_com.FULL_QUOTE != "" ]
    
    jb = pd.concat( [ jb_plen, jb_com ] )
    
    # Salva
    directory = "../data/csvs/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    fname = directory + "discursos_jair_bolsonaro.csv" 
    jb.to_csv(fname, index = False)
    
    return jb

In [37]:
run_script()

,CONTENT,FILE,SESSION_DATE,SESSION_TYPE,SPEAKER_COUNT,FULL_QUOTE
0,O SR. JAIR BOLSONARO (PP-RJ. Sem revisao do or...,../data/txts/plenario/14122010-15h32-BREVES CO...,2010-12-14,BREVES COMUNICACOES,1,"Sr. Presidente, Sras. e Srs. Deputados, primei..."
1,O SR. JAIR BOLSONARO (PP-RJ. Sem revisao do or...,../data/txts/plenario/21092011-13h58-BREVES CO...,2011-09-21,BREVES COMUNICACOES,1,"Sr. Presidente, Sras. e Srs. Deputados, mais u..."
2,O SR. JAIR BOLSONARO (PP-RJ. Sem revisao do or...,../data/txts/plenario/17062009-16h18-ORDEM DO ...,2009-06-17,ORDEM DO DIA,1,"Sr. Presidente, talvez eu seja o unico Deputad..."
3,O SR. JAIR BOLSONARO (PTB-RJ. Sem revisao do o...,../data/txts/plenario/10112004-12h00-BREVES CO...,2004-11-10,BREVES COMUNICACOES,1,"Sr. Presidente, Sras. e Srs. Deputados, todos ..."
4,O SR. JAIR BOLSONARO (PP-RJ. Sem revisao do or...,../data/txts/plenario/26102005-14h26-PEQUENO E...,2005-10-26,PEQUENO EXPEDIENTE,1,"Sr. Presidente, meu prezado Deputado Inocencio..."
5,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisao do o...,../data/txts/plenario/29112000-15h18-BREVES CO...,2000-11-29,BREVES COMUNICACOES,1,"Sr. Presidente, Sras. e Srs. Deputados, vou fa..."
6,O SR. JAIR BOLSONARO (PP-RJ. Pela ordem. Sem r...,../data/txts/plenario/01122010-14h26-PEQUENO E...,2010-12-01,PEQUENO EXPEDIENTE,1,"Sr. Presidente, na semana passada por tres dia..."
7,O SR. JAIR BOLSONARO (PP-RJ. Pela ordem. Sem r...,../data/txts/plenario/30042013-10h30-BREVES CO...,2013-04-30,BREVES COMUNICACOES,1,"Sr. Presidente, no ultimo domingo, no SBT, a a..."
8,O SR. JAIR BOLSONARO (PTB-RJ. Sem revisao do o...,../data/txts/plenario/13032003-14h18-PEQUENO E...,2003-03-13,PEQUENO EXPEDIENTE,1,"a Sr. Presidente, abordarei assunto deveras po..."
9,O SR. JAIR BOLSONARO (Bloco/PP-RJ. Pela ordem....,../data/txts/plenario/06112013-16h06-BREVES CO...,2013-11-06,BREVES COMUNICACOES,1,"E muito tempo, Sr. Presidente. Meus companheir..."
